In [1]:
# Machine learning pipeline for Doench Data, supplementary table 7
import pandas as pd

supp_7_path = "/Users/daveistanto/Dropbox/UIUCGraduateSchool/Researches/CROPSR_head_dir/data_files/supp_table_7.csv"
supp_7_df = pd.read_csv(supp_7_path)

In [2]:
# # Sort based on Gene % Rank but group by Gene
import warnings
warnings.filterwarnings('ignore')
unique_gene_list = list(set(supp_7_df["Gene"]))

supp_7_df["Top 20%"] = False

for gene in unique_gene_list:
    supp_7_df_temp = supp_7_df[supp_7_df["Gene"] == gene]
    supp_7_df_temp["Top 20%"][0:int(len(supp_7_df_temp)*0.2)] = True
    supp_7_df[supp_7_df["Gene"] == gene] = supp_7_df_temp
    

In [3]:
# Extract features

import Feature_Extraction as fe

feat_vec = supp_7_df["Expanded Sequence"].apply(fe.ext_sgRNA_feat)

print(supp_7_df.columns)

Index(['Sequence', 'Expanded Sequence', 'Position', 'Type', 'Gene',
       'Transcript', 'Strand', 'Gene % Rank', 'sgRNA Score', 'Top 20%'],
      dtype='object')


In [4]:
# Split dataset to 80% Training 20% Test
from sklearn.model_selection import train_test_split 
import numpy as np

vec_train, vec_test, label_train, label_test = train_test_split(feat_vec.values, supp_7_df["Top 20%"].values, test_size=0.2, random_state=0)
# Convert nested np.array to 2d np.array
vec_train = np.array(list(vec_train))
vec_test = np.array(list(vec_test))

In [13]:
# Eliminate most features: SVM with L-1 regularization constant

from sklearn.svm import LinearSVC

SVM_clf = LinearSVC(penalty='l1', loss='squared_hinge', dual=False)
SVM_clf.fit(vec_train, label_train)

# Function for filtering, make vectorized version

def filter_vec(input_vec, filter_bool):
    return input_vec[filter_bool[0]]

# Filter for non-0 weights, and only use corresponding features

non_0_filter = (SVM_clf.coef_ != 0)
non_0_weights = SVM_clf.coef_[non_0_filter]
filtered_vec_train = np.apply_along_axis(filter_vec, 1, vec_train, non_0_filter)
filtered_vec_test = np.apply_along_axis(filter_vec, 1, vec_test, non_0_filter)

print("Original # of coefficient:", len(SVM_clf.coef_[0]))
print("Non 0 # of coefficient", len(non_0_weights))

Original # of coefficient: 590
Non 0 # of coefficient 303


In [14]:
# Linear neural network Classifier, ranked

import torch
from torch import nn

prob_treshold = 0.7
class nnClassifier(nn.Module):
    def __init__(self):
        super(nnClassifier, self).__init__()
        
        self.lin_1 = nn.Linear(590, 300)
        self.lin_2 = nn.Linear(300, 100)
        self.lin_3 = nn.Linear(100, 10)
        self.lin_4 = nn.Linear(10, 1)
        
        self.seq = nn.Sequential(
            self.lin_1,
            nn.ReLU(),
            self.lin_2,
            nn.ReLU(),
            self.lin_3,
            nn.ReLU(),
            self.lin_4,
            nn.Sigmoid()
        )

    def forward(self, x):
#         print(x.type(torch.FloatTensor))
        x = self.seq(x)
#         label = 
#         print(x)
#         if x >= 0.7:
            
        
        return x

In [21]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print (x > 3)
    

tensor([[0, 0, 0],
        [1, 1, 1]], dtype=torch.uint8)
